# Check: initial downloads

Make maps to see that rail/ferry/brt all show up correctly.

In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

import geopandas as gpd
import pandas as pd

from IPython.display import Markdown

from update_vars import analysis_date

# Map arguments
TILES = "Carto DB Positron"

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
def make_map(gdf, plot_col):
    if "service_date" in gdf.columns:
        gdf = gdf.drop(columns = "service_date")
        
    m = gdf.explore(plot_col, categorical = True, tiles = TILES)
    
    display(m)

## After `A1_download_rail_ferry_brt`

* There are some stops to remove. 
* Once finalized, can run `A2_combine_stops`

In [3]:
import A1_download_rail_ferry_brt_stops as rail_ferry_brt

### LA Metro (182)

In [4]:
metro_brt_stops = rail_ferry_brt.grab_operator_brt(182, analysis_date)

In [5]:
make_map(metro_brt_stops, "route_id")

In [6]:
from A2_combine_stops import metro_street_running

metro_brt_stops2 = rail_ferry_brt.additional_brt_filtering_out_stops(
    metro_brt_stops, 182, metro_street_running)

In [7]:
make_map(metro_brt_stops2, "route_id")

### SF Muni (282)

In [8]:
muni_brt_stops = rail_ferry_brt.grab_operator_brt(282, analysis_date)

In [9]:
make_map(muni_brt_stops, "route_id")

In [10]:
from A2_combine_stops import van_ness_ids

muni_brt_stops2 = rail_ferry_brt.additional_brt_filtering_out_stops(
        muni_brt_stops, 282, van_ness_ids)

In [11]:
make_map(muni_brt_stops2, "route_id")

### AC Transit (4)

In [12]:
act_brt_stops = rail_ferry_brt.grab_operator_brt(4, analysis_date)

In [13]:
make_map(act_brt_stops, "route_id")

## After `A3_rail_ferry_brt_extract`

#### BRT Service likely meeting [PRC 21060.2](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PRC&sectionNum=21060.2.&highlight=true&keyword=bus%20rapid%20transit) definition:

* LA Metro Orange, Silver excluding street running (stop flags only)
* ~~Omnitrans sbX, all stops (curbside stations are well-defined, with fare prepayment)~~
    * insufficient frequency 5/16
* AC Transit Tempo, all stops (curbside stations are well-defined, with fare prepayment)

In [14]:
import A3_rail_ferry_brt_extract as A3

stops = A3.get_rail_ferry_brt_extract()

In [15]:
hqta_types = list(stops.hqta_type.unique().compute())

In [16]:
for i in hqta_types:
    subset = stops[stops.hqta_type==i].compute()
    
    display(Markdown(f"### HQTA Type: {i}"))
    
    make_map(subset, "calitp_itp_id_primary")

### HQTA Type: major_stop_rail

### HQTA Type: major_stop_brt

### HQTA Type: major_stop_ferry